In [58]:
import jet 
import numpy as np

### Approach #1 (Explicitly Defined Gates)

In [59]:
# Define all gates for test circuit
angle = (np.pi/4)*1j
angle_p = -(np.pi/4) * 1j
T = jet.Tensor(["8","9"], [2,2], [1,0,0,np.exp(angle)])
T_p = jet.Tensor(["0","1"], [2,2],[1,0,0,np.exp(angle_p)])

# Defining CNOT gate
# We can view it as a rank-4 tensor of size 2 by 2 by 2 by 2
# Two of the legs of this tensor represent inputs and 
# the other two legs represent outputs
CNOT = jet.Tensor(["7", "4", "8", "10"], [2, 2, 2, 2])
CNOT.set_value((0, 0, 0, 0), 1) # |00> -> |00>
CNOT.set_value((0, 1, 0, 1), 1) # |01> -> |01>
CNOT.set_value((1, 0, 1, 1), 1) # |10> -> |11>
CNOT.set_value((1, 1, 1, 0), 1) # |11> -> |10>

CNOT_p = jet.Tensor(["1", "3", "2", "4"], [2, 2, 2, 2])
CNOT_p.set_value((0, 0, 0, 0), 1) # |00> -> |00>
CNOT_p.set_value((0, 1, 0, 1), 1) # |01> -> |01>
CNOT_p.set_value((1, 0, 1, 1), 1) # |10> -> |11>
CNOT_p.set_value((1, 1, 1, 0), 1) # |11> -> |10>

# Defining Hadamard gate
inv_sqrt_2 = 2 ** -0.5
# The Hadamard gate can be viewed as a rank-2 tensor of size 2 by 2
H = jet.Tensor(["6", "7"], [2, 2], [inv_sqrt_2, inv_sqrt_2, inv_sqrt_2, -inv_sqrt_2])
H_p = jet.Tensor(["2", "5"], [2, 2], [inv_sqrt_2, inv_sqrt_2, inv_sqrt_2, -inv_sqrt_2])

# Defining Pauli-Z Gate
pauli_z_data = [1, 0, 0, -1]
size = [2,2]
indices = ["5","6"]
Z = jet.Tensor(indices, size, pauli_z_data)

tn = jet.TensorNetwork();
tn.add_tensor(T_p)
tn.add_tensor(CNOT_p)
tn.add_tensor(H_p)
tn.add_tensor(Z)
tn.add_tensor(H)
tn.add_tensor(CNOT)
tn.add_tensor(T)
result = tn.contract()
result = np.reshape(result.data, [4,4])
print(result.real)
print(result.imag * 1j)

[[0.         0.         0.         0.70710678]
 [0.         0.70710678 0.         0.        ]
 [0.         0.         0.70710678 0.        ]
 [0.70710678 0.         0.         0.        ]]
[[ 0.+0.j          0.+0.j          0.+0.j          0.+0.70710678j]
 [ 0.+0.j          0.+0.70710678j  0.+0.j          0.+0.j        ]
 [ 0.+0.j          0.+0.j         -0.-0.70710678j  0.+0.j        ]
 [-0.-0.70710678j  0.+0.j          0.+0.j          0.+0.j        ]]


The above result appears to be incorrect (requires debugging)

### Approach #2 (Creating own gate using Jet QubitGate Class)

In [60]:
from functools import lru_cache

INV_SQRT2 = 1 / np.sqrt(2)
angle = (np.pi/4)*1j

class UGate(jet.QubitGate):
    """ UGate represents our QCSAT instance """

    def __init__(self):
        super().__init__(name="UGate", num_wires=2)

    @lru_cache()
    def _data(self):
        I = np.array([[1,0],[0,1]])
        T = np.array([[1,0],[0,np.exp(angle)]])
        CNOT = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]])
        H = np.array([[INV_SQRT2, INV_SQRT2], [INV_SQRT2, -INV_SQRT2]])
        mat1 = np.kron(T,I)
        mat2 = CNOT
        mat3 = np.kron(H,I)
        mat = np.matmul(mat3, mat2)
        mat = np.matmul(mat, mat1)
        return mat

In [62]:
tn = jet.TensorNetwork()
U_tr = jet.Adjoint(UGate()).tensor()
U_tr.rename_index(0, '0')
U_tr.rename_index(1, '2')
U_tr.rename_index(2, '5')
U_tr.rename_index(3, '4')
U = UGate().tensor()
U.rename_index(0, '6')
U.rename_index(1, '4')
U.rename_index(2, '9')
U.rename_index(3, '10')
Z = jet.PauliZ().tensor()
Z.rename_index(0, '5')
Z.rename_index(1, '6')
tn.add_tensor(U_tr)
tn.add_tensor(Z)
tn.add_tensor(U)
result = tn.contract()
result = np.reshape(result.data, [4,4])
print(result.real)
print(result.imag * 1j)

[[0.         0.         0.         0.70710678]
 [0.         0.         0.70710678 0.        ]
 [0.         0.70710678 0.         0.        ]
 [0.70710678 0.         0.         0.        ]]
[[ 0.+0.j          0.+0.j          0.+0.j          0.+0.70710678j]
 [ 0.+0.j          0.+0.j          0.+0.70710678j  0.+0.j        ]
 [ 0.+0.j         -0.-0.70710678j  0.+0.j          0.+0.j        ]
 [-0.-0.70710678j  0.+0.j          0.+0.j          0.+0.j        ]]


The above matrices reflect the desired result